#  Algoritmo Genético Rastros
## Projeto nº 3
### Introdução à Inteligência Artificial edição 2020/21


## Grupo: 22

### Elementos do Grupo

Nome: André Firmino

Número: 44999

Nome: Joao Janeiro

Número: 52779

Nome: Nuno Estalagem

Número 52828

In [15]:
from copy import copy,deepcopy
from rastros import*
from utils import *
from jogos import*
from search import* 
from geneticSolo import *
import random
import string
import math    
import time

### Função Projeto 2

Esta função já provém da fase anterior do projeto,
em que se atribui uma pontuação a cada momento jogada de um jogador.
A primeira condição diz respeito a uma condição de vitória, sendo que o vencedor é o jogador que se dirige para Sul.
A segunda condição diz respeito a uma condição de vitória, sendo que o vencedor é o jogador que se dirige para Norte.
A terceira condição diz respeito a uma condição em que ainda não se verifica vitória, sendo que cada jogador através das funções calcScoreWinS e calcScoreWinN, vai avaliando a melhor forma de jogar, dado um estado e os movimentos que pode realizar.

In [16]:
def win_Basilio(jogador,estado):
    movimentos=estado.moves()
    if estado.terminou == 1:
        return 100 if jogador == "S" else -100
    elif estado.terminou == -1:
        return 100 if jogador == "N" else -100
    else: 
        if jogador=="N":
            return calcScoreWinN(estado,movimentos,(1,8)) #formaPretasN(estado)
        else:
            return calcScoreWinS(estado, movimentos, (8,1))

### Funções Avaliação Projeto 2
Tanto uma função como outra, apesar de realizarem com direções complementares,
atribuem um dado número de pontos a uma direção que a peça pode seguir, que aumenta quanto mais vantajosa
for essa jogada para o jogador em questão.

In [17]:
def  calcScoreWinS(estado,moves,win):
    calc=0
    blacks=list(estado.blacks)
    white=estado.white         
    calc-=distancia(white,(8,1))  
    
    if (white[0]+1,white[1]-1) in moves:
        calc+=10     
    elif  (white[0]+1,white[1]) in moves:
        calc+= 7
    elif (white[0],white[1]-1) in moves:
        calc+=3
    elif (white[0]-1,white[1]-1) in moves or (white[0]+1,white[1]+1) in moves:
        calc+= 0 
    elif ((white[0]-1,white[1]) in moves) or ((white[0],white[1]+1) in moves):
        calc -= 3
    elif (white[0]-1,white[1]+  1) in moves:
        calc-=10
    else:
        calc+= 1
    return calc
    
    
def  calcScoreWinN(estado,moves,win): 
    calc=0
    blacks=list(estado.blacks)
    white=estado.white         
    calc-=distancia(white,(1,8))
 
    if (white[0]-1,white[1]+1) in moves:
        calc+=10
    elif  (white[0]-1,white[1]) in moves:
        calc+= 7  
    elif (white[0],white[1]+1) in moves:
        calc+=3  
    elif (white[0]+1,white[1]+1) in moves or (white[0]-1,white[1]-1) in moves:
        calc+= 0 
    elif ((white[0]+1,white[1]) in moves) or ((white[0],white[1]-1) in moves):
        calc -= 3 
    elif (white[0]+1,white[1]-1) in moves:
        calc-=10
    else:
        calc+= 1
    return calc

### Funções Avaliação Fornecidas
Tanto uma função de avaliação como outra são fornecidas pelos professores,
sendo que a primeira diz respeito a avaliar consoante o objetivo de cada jogador,
enquanto que a segunda é relativa avaliar consoante o objetivo oposto de cada jogador.

In [18]:
def dist_Obj(jogador,estado):
    if jogador=="S":
        obj=(8,1)
    else:
        obj=(1,8)
    return distancia(estado.white,obj)

def dist_Base(jogador,estado):
    if jogador=="S":
        obj=(1,8)
    else:
        obj=(8,1)
    return distancia(estado.white,obj)

### Classe Jogador - Criação Jogador
Esta classe permite a criação de uma instância de jogador, 
que será útil para a realização do algoritmo coevolucionário
Tem como atributos *nome* (nome do jogador), *pesos* 
(peso a atribuir a cada funcao de avaliacao) e *funcs* 
(funcoes de avaliação que serão usadas)
As funções existentes são *display*, que nos apresenta uma representação
textual do jogador e *aval_pesos*, que, dado um estado, avalia os pesos
de realizar as funções de avaliação e toma a ação em concordância.

In [19]:
class Jogador():
    def __init__(self, nome, pesos,funcs):
        self.nome = nome
        self.pesos =pesos
        self.funcs=funcs
        self.depth=1
        self.fun=lambda game,state:alphabeta_cutoff_search_new(state,game,self.depth,eval_fn=self.aval_pesos)
    def display(self):
        print(nome+" ")
    def aval_pesos(self,estado,jogador):
        total=0
        for i in range(len(self.funcs)):
            total+=self.funcs[i](jogador,estado)*self.pesos[i]
        return total

### Torneio
Função que, dada um k arbitrário e um conjunto de indivíduos,
vai selecionar k indivíduos aleatoriamente e, desses,
escolhe o melhor. Este método é invocado duplamente, 
uma vez para escolher o pai e outra para escolher a mãe.

In [20]:
def torneioDeK(ordered_dicionario,players,k,escolhidos_elitismo):
    listmaxs=[]
    for i in range(2):
        k_selected_prov=deepcopy(players)
        for x in listmaxs:
            for y in k_selected_prov:
                if y.nome==x.nome:  
                    k_selected_prov.remove(y)
        k_selected_players=[]
        for x in range(k):
            rand=random.choice(k_selected_prov)
            k_selected_prov.remove(rand)
            k_selected_players.append(rand)
        del k_selected_prov
        currMax=-1
        curr_second_max=-1
        curr_best_player=k_selected_players[0]
        for x in k_selected_players:
            if ordered_dicionario[x.nome]>currMax:
                curr_best_player=x
                currMax=ordered_dicionario[x.nome]
        listmaxs.append(curr_best_player)
    print("escolhidos para K:")
    i=0
    for x in listmaxs:
        print(str(i)+": " + str(x.pesos) )
        i+=1
    return listmaxs  

### Combinação Progenitores
Em termos biológicos, corresponde ao momento
da procriação, em que os progenitores combinam
os seus genes, gerando descendência. Esta 
descendência é caracterizada por corresponder 2 filhos,
em que ambos têm um ponto de cruzamento aleatório.
Em cada um dos filhos, dados k o tamanho dos indivíduos,
e n o indice de cruzamento. têm k-n genes de um dos pais antes
do ponto de cruzamento e k-n genes do outro dos pais, depois 
do ponto de cruzamento.

In [21]:
def recombine_new(array,x, y):
    n = len(x)
    c = random.randrange(0, n)
    array.append(x[:c] + y[c:])
    array.append(y[:c]+x[c:])

### Elitismo
Função que pretende procurar o melhor indivíduo de cada geração
na geração seguinte. Isto é feito, tendo em conta uma percentagem de
elitismo (*elit*, que vai determinar quão restrita queremos a nossa elite, um valor
entre 0 e 1) e os fitnesses da taça, escolhe os *elit * número total de indivíduos*
melhores jogadores.

In [22]:
def elitism(elit,players,ordered_dicionario):
    elitismo= elit*len(players)
    if (math.floor(elitismo))%2==0:
        elitismo=(math.floor(elitismo))
    else:
        elitismo=(math.ceil(elitismo))
    temp=elitismo
    next_gen=[]
    for x in reversed(ordered_dicionario):
        if temp!=0:
              for y in players:
                if y.nome==x:
                    next_gen.append(y)
                    temp-=1
                    break
        else:
            del temp
            break
    return next_gen


### Mutação
Função que vai possibilitar a realização aleatória 
(com uma dada probabilidade de mutação dada como argumento)
de uma mutação num dado gene da pool de genes.

In [23]:
def mutate(x, gene_pool, pmut):
    for weight in range(len(x)):
        if random.uniform(0, 1) >= pmut:
            continue
        else:
            print("O peso: " +str(x[weight]) + " mutou")
            x[weight]=random.choice(gene_pool)

    return x

### Função Coevolução
Função que vai permitir a realização da evolução em si,
dada uma determinada geração com uma certa dimensão, 
e que verifica todas as alíneas verificadas na secção
"Teste de Programas", sendo de se destacar a criação
da geração inicial (*population=init_population(dim,gene_pool,len(caracts))*), 
a Seleção por Torneio (*fitness_taca(players,jogo,dicionario,3)*) e 
a geração de dois descendentes (*array.append(recombine_new(prov_population,guys[0],guys[1]))*). 
Além disso, é possível a concretização de uma mutação (), a Evolução
do Torneio Tipo Taça (através das sucessivas iterações no ciclo While)
e ainda o recurso ao Elitismo (*escolhidos_elitismo=elitism(elit,players,ordered_dicionario)*).


In [24]:
def coevol(gen,dim,gene_pool=range(-20,20),elit=0.2,p_mut=0.1,k_rivais=2,lim_profs=[4,5],jogo= Rastros,caracts = [dist_Base,dist_Obj,win_Basilio],numJogos=6
):
    k_rivais=int(dim/3)
    population=init_population(dim,gene_pool,len(caracts))
    players=[]
    letrasm=string.ascii_lowercase
    letrasM=string.ascii_uppercase
    letras_tot=letrasM+letrasm
    for guy in population:
        novoPlayer = Jogador(''.join(random.choice(letras_tot) for i in range(10)),guy,caracts)
        players.append(novoPlayer)
    
    for x in players:
        print("jogador " + x.nome + "  pesos: " + str(x.pesos) + "\n")
        
      
    i=0
    elitMax=[]
    while i<gen-1:
    
        dicionario={}
        for x in players:
            dicionario[x.nome]=1
        fitness_taca(players,jogo,dicionario,numJogos)
        ordered_dicionario = {}
        sorted_keys = sorted(dicionario, key=dicionario.get)  # [1, 3, 2]

        for w in sorted_keys:
            ordered_dicionario[w] = dicionario[w]
            
        escolhidos_elitismo=elitism(elit,players,ordered_dicionario)
        print("elitismo: ")
        b=1
        for x in escolhidos_elitismo:
            print(str(b)+": " + str(x.pesos))
            b+=1
    
        parents=torneioDeK(ordered_dicionario,players,k_rivais,escolhidos_elitismo)
        guys=[]
        for parent in parents:
            guys.append(parent.pesos)
        for x in guys:
            print("futuro pai/mae " + str(x))
        print("\n")
        prov_population=[]
        array=[]
        while (len(prov_population)+len(escolhidos_elitismo)) != dim:
            recombine_new(prov_population,guys[0],guys[1])
        for x in prov_population:
            mutate(x,gene_pool,p_mut)
    
        for x in escolhidos_elitismo:
            prov_population.append(x.pesos)
        del players
        players=[]
        for guy in prov_population:
            novoPlayer = Jogador(''.join(random.choice(letrasm+letrasm) for i in range(10)),guy,caracts)
            players.append(novoPlayer)            
        i+=1
    
    elitMax=[]
    dicionario={}
    for x in players:
        dicionario[x.nome]=1
    fitness_taca(players,jogo,dicionario,numJogos)
    ordered_dicionario = {}
    sorted_keys = sorted(dicionario, key=dicionario.get)  # [1, 3, 2]

    for w in sorted_keys:
        ordered_dicionario[w] = dicionario[w]
    escolhidos_elitismo=elitism(elit,players,ordered_dicionario)
    
       
    elitMax=escolhidos_elitismo[0].pesos
    print("Eis os pesos obtidos: " +str(elitMax))
    return elitMax

### Jogo
Dados dois jogadores, jog1 e jog2, que possuem funções
intrinsecamente, devolvem o conjunto de jogadas que realizam 
no decorrer do jogo para o procurarem ganhar.
Enquanto que o jogo não tenha alcançado o seu estado
terminal, vão sendo adicionadas jogadas, para cada jogador,
à lista de jogadas, sendo esta retornada quando o algoritmo é chamado.

In [25]:
def jogar(jog1, jog2,jogo):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve uma lista de jogadas e o resultado 1 se S ganha
    game =jogo() 
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    while not game.terminal_test(estado):
        jogada = proxjog.fun(game, estado)
        estado=game.result(estado,jogada)
        lista_jogadas.append((proxjog.nome, jogada))
        proxjog = jog2 if proxjog == jog1 else jog1
    return  estado.terminou

### Cálculo Fitness
Função que permite criar um torneio entre diversos
indivíduos e que devolve os valores
*fitness* de cada indivíduo participante no torneio.

In [26]:
def fitness_taca(jogadores,game,dicionario,numJogos):
    localDic={}
    for x in jogadores:
        localDic[x.nome]=0
    lista_nao_verificaveis=[]
    lista_jogos=[]
    lista_jogadores_nao_emparelhados=deepcopy(jogadores)
    count=0
    while len(lista_jogadores_nao_emparelhados)!=0:
        jog=random.choice(lista_jogadores_nao_emparelhados)
        lista_jogadores_nao_emparelhados.remove(jog)
        adv=random.choice(lista_jogadores_nao_emparelhados)
        lista_jogadores_nao_emparelhados.remove(adv)
        
        
        jog.depth=1
        jog.fun=lambda game,state:alphabeta_cutoff_search_new(state,game,jog.depth,eval_fn=jog.aval_pesos)
        adv.depth=1
        adv.fun=lambda game,state:alphabeta_cutoff_search_new(state,game,adv.depth,eval_fn=adv.aval_pesos)
        for x in range(0,int(numJogos/2)):
            lista_jogos.append((jog.nome, adv.nome,jogar(jog,adv, game)))
            lista_jogos.append((adv.nome,jog.nome,jogar(adv,jog, game)))
         
        jog.depth=2
        jog.fun=lambda game,state:alphabeta_cutoff_search_new(state,game,jog.depth,eval_fn=jog.aval_pesos)
        adv.depth=2
        adv.fun=lambda game,state:alphabeta_cutoff_search_new(state,game,jog.depth,eval_fn=adv.aval_pesos)
        
        for x in range(0,numJogos-int(numJogos/2)):
            lista_jogos.append((jog.nome, adv.nome,jogar(jog,adv, game)))
            lista_jogos.append((adv.nome,jog.nome,jogar(adv,jog, game)))

        
    for x in lista_jogos:
        if x[2]==1:
            localDic[x[0]]+=1
        else:
            localDic[x[1]]+=1
    for x in localDic:
        for y in jogadores:
            if x == y.nome:
                print("peso " + str(y.pesos) + "   wins: " + str(localDic[x])  )
                break
    novaLista=[]
    for x in localDic:
        if math.floor((localDic[x]))>math.floor((numJogos)):     
            for jogador in jogadores:
                if jogador.nome==x:
                    novaLista.append(jogador)
                    break
        if math.floor((localDic[x]))==math.floor(numJogos) and x not in lista_nao_verificaveis:
            nome=""
            for jogo in lista_jogos:
                if jogo[0] == x or jogo[1]==x:
                    nome=random.choice([jogo[0],jogo[1]])
                    lista_nao_verificaveis.append(jogo[1])
                    lista_nao_verificaveis.append(jogo[0])
                    break
            for jogador in jogadores:
                if jogador.nome==nome:
                    novaLista.append(jogador)
                    break
   
    for x in novaLista:
        dicionario[x.nome]+=1
    if len(novaLista)!=1:
        print("\n")
        return fitness_taca(novaLista, game, dicionario,numJogos)
    return dicionario

#### Exemplo de aplicação

In [27]:
p=coevol(5, 32)

jogador FPWbTXlFYQ  pesos: [-6, 12, 4]

jogador TLprJLMlpU  pesos: [14, 11, -2]

jogador gdFLWVoQji  pesos: [-13, 19, -8]

jogador xQdutszdnt  pesos: [-11, 5, 2]

jogador klSdlLKddV  pesos: [-2, -10, 12]

jogador OAbZpZHYjx  pesos: [-9, 5, 14]

jogador XmtDxgunoW  pesos: [-15, -17, 12]

jogador qYzQljrEEO  pesos: [8, -16, -16]

jogador GtECFMyFOC  pesos: [-14, 3, 1]

jogador UjiNinzgcw  pesos: [-14, -7, -6]

jogador QFPEzpwZVJ  pesos: [-11, 6, 16]

jogador MvPXnARWcU  pesos: [13, 8, -7]

jogador AEgFyIHmXK  pesos: [11, -6, 8]

jogador PATcpHMSTj  pesos: [-6, 8, 7]

jogador xCwUWivocA  pesos: [11, -18, 6]

jogador KnBKTdesBI  pesos: [9, 0, -4]

jogador IvShzdCzXX  pesos: [5, 10, 15]

jogador DkYJheNDto  pesos: [-9, -11, -13]

jogador zDKTxCYmYb  pesos: [-3, 15, 9]

jogador beEJqUQxaj  pesos: [-1, -13, 18]

jogador ujudCOzxdy  pesos: [-7, 0, -15]

jogador DFkZTsFWTy  pesos: [2, -6, 10]

jogador xhqYUkkFGX  pesos: [0, 16, -19]

jogador YBzfbPnhZs  pesos: [4, 15, -10]

jogador PohxUbYcQu  

peso [17, 19, 14]   wins: 6
peso [12, 19, 14]   wins: 8
peso [17, 19, 14]   wins: 4
peso [12, 19, 14]   wins: 6
peso [12, 19, 9]   wins: 4
peso [17, 19, 14]   wins: 6
peso [12, 19, 14]   wins: 9
peso [17, 19, 14]   wins: 6
peso [12, 19, 14]   wins: 6
peso [12, 19, 14]   wins: 6
peso [12, 19, 14]   wins: 3
peso [17, 19, 14]   wins: 6
peso [12, -6, 14]   wins: 3
peso [12, 19, 14]   wins: 6
peso [17, 9, 5]   wins: 9
peso [17, 9, 5]   wins: 8


peso [17, 19, 14]   wins: 1
peso [12, 19, 14]   wins: 5
peso [12, 19, 14]   wins: 8
peso [17, 19, 14]   wins: 3
peso [12, 19, 14]   wins: 11
peso [17, 19, 14]   wins: 4
peso [17, 9, 5]   wins: 9
peso [17, 9, 5]   wins: 7


peso [12, 19, 14]   wins: 8
peso [12, 19, 14]   wins: 7
peso [17, 9, 5]   wins: 4
peso [17, 9, 5]   wins: 5


peso [12, 19, 14]   wins: 4
peso [12, 19, 14]   wins: 8
elitismo: 
1: [12, 19, 14]
2: [12, 19, 14]
3: [17, 9, 5]
4: [17, 9, 5]
5: [17, 19, 14]
6: [17, 19, 14]
escolhidos para K:
0: [17, 19, 14]
1: [12, 19, 14]
futuro pai/m